In [1]:
import sys
sys.path.append('./base_model')
from interface import PolyDisVAE
import torch, os, numpy as np

In [2]:
# initialize the VAE model
polydis_model = PolyDisVAE.init_model()
polydis_model.load_model('./model_param/polydis-v1.pt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/workspace/vae-tuning-texutres/interface.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dic = torch.load(model_path, map_location=self.device)


In [3]:
# prepare feature files and output location
from glob import glob
feature_dir = './features'
latent_dir = './latent_features'
os.makedirs(latent_dir, exist_ok=True)
feature_files = sorted(glob(os.path.join(feature_dir, '*.npz')))
len(feature_files)

62034

In [4]:
def encode_block(files):
    z_chds = []
    z_txts = []
    for path in files:
        with np.load(path) as feat:
            pr = feat['piano_roll']
            chd = feat['chord']
        c = torch.from_numpy(chd).float().to(device).unsqueeze(0)
        pr = torch.from_numpy(pr).float().to(device).unsqueeze(0)
        z_chd = polydis_model.chd_encode(c)
        z_txt = polydis_model.txt_encode(pr)
        z_chds.append(z_chd.cpu().numpy())
        z_txts.append(z_txt.cpu().numpy())
    return np.concatenate(z_chds), np.concatenate(z_txts)

In [5]:
# process files in blocks and store latent codes
block_size = 100
for start in range(0, len(feature_files), block_size):
    block = feature_files[start:start + block_size]
    z_chd, z_txt = encode_block(block)
    out_path = os.path.join(latent_dir, f'latents_{start//block_size:03d}.npz')
    np.savez(out_path, z_chd=z_chd, z_txt=z_txt, files=block)
    print(f'processed {start + len(block)} / {len(feature_files)} files')

processed 100 / 62034 files
processed 200 / 62034 files
processed 300 / 62034 files
processed 400 / 62034 files
processed 500 / 62034 files
processed 600 / 62034 files
processed 700 / 62034 files
processed 800 / 62034 files
processed 900 / 62034 files
processed 1000 / 62034 files
processed 1100 / 62034 files
processed 1200 / 62034 files
processed 1300 / 62034 files
processed 1400 / 62034 files
processed 1500 / 62034 files
processed 1600 / 62034 files
processed 1700 / 62034 files
processed 1800 / 62034 files
processed 1900 / 62034 files
processed 2000 / 62034 files
processed 2100 / 62034 files
processed 2200 / 62034 files
processed 2300 / 62034 files
processed 2400 / 62034 files
processed 2500 / 62034 files
processed 2600 / 62034 files
processed 2700 / 62034 files
processed 2800 / 62034 files
processed 2900 / 62034 files
processed 3000 / 62034 files
processed 3100 / 62034 files
processed 3200 / 62034 files
processed 3300 / 62034 files
processed 3400 / 62034 files
processed 3500 / 62034 

KeyboardInterrupt: 

In [ ]:
# example of reading saved latent representations later
latent = np.load(os.path.join(latent_dir, 'latents_000.npz'))
z_chd = latent['z_chd']
z_txt = latent['z_txt']
file_names = latent['files']
print(z_chd.shape, z_txt.shape)